In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(H5weaver)
library(hise)
library(tidyverse)
library(SeuratDisk)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: Matrix

Loading required package: rhdf5


Attaching package: ‘H5weaver’


The following objects are masked from ‘package:rhdf5’:

    h5dump, h5ls


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.3     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ 

#### 3prime

In [2]:
b077_h5s <- c(
    '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/B077_3prime/B077-P2_PB00607-02_2021-07-23T23_25_52.724598929Z_labeled.h5',
    '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/B077_3prime/B077-P2_PB00609-02_2021-07-23T23_25_52.724598929Z_labeled.h5',
    '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/B077_3prime/B077-P2_PB00610-02_2021-07-23T23_25_52.724598929Z_labeled.h5'
)

In [3]:
# library(H5weaver)
mat <- read_h5_dgCMatrix(b077_h5s[[1]])

rownames(mat) <- make.unique(rownames(mat))

cell_meta <- read_h5_cell_meta(b077_h5s[[1]])

so_v3_1 <- CreateSeuratObject(mat, meta.data = cell_meta, project='PB00607')



mat <- read_h5_dgCMatrix(b077_h5s[[2]])

rownames(mat) <- make.unique(rownames(mat))

cell_meta <- read_h5_cell_meta(b077_h5s[[2]])

so_v3_2 <- CreateSeuratObject(mat, meta.data = cell_meta, project='PB00609')


mat <- read_h5_dgCMatrix(b077_h5s[[3]])

rownames(mat) <- make.unique(rownames(mat))

cell_meta <- read_h5_cell_meta(b077_h5s[[3]])

so_v3_3 <- CreateSeuratObject(mat, meta.data = cell_meta, project='PB00610')

In [4]:
v3_fully <- merge(so_v3_1, c(so_v3_2, so_v3_3))

#### flex

In [5]:
flex <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/exp340/Exp00340_w7/', 
    pattern = 'filtered_feature_bc_matrix.h5$', 
    full.names = TRUE, 
    recursive = TRUE
)

In [6]:
flex <- lapply(flex, function(h5){
    x <- h5
    x <- strsplit(x,'/outs/')[[1]][2]
    l <- length(strsplit(x ,'/')[[1]])
    x <- strsplit(x ,'/')[[1]][l-2]
    if (x %in% c('PB00607-06-01','PB00609-06-01','PB00610-06-01')){
        mtx <- Read10X_h5(h5)
        so <- CreateSeuratObject(mtx,project=x)
        return(so)
    }
    })

In [7]:
flex[sapply(flex, is.null)] <- NULL


In [8]:
flex_fully <- Reduce(merge, flex)

v3_fully <- JoinLayers(v3_fully)
flex_fully <- JoinLayers(flex_fully)

### MT Filtering

In [9]:
v3_fully[["percent.mt"]] <- PercentageFeatureSet(v3_fully, pattern = "^MT-")
flex_fully[["percent.mt"]] <- PercentageFeatureSet(flex_fully, pattern = "^MT-")


In [10]:
v3_fully   <- v3_fully   %>% subset(subset = percent.mt < 5)
flex_fully <- flex_fully %>% subset(subset = percent.mt < 5)

In [11]:
v3_fully <- NormalizeData(v3_fully) %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>% 
    RunUMAP(dims = 1:20) %>% 
    FindNeighbors(dims = 1:20) %>% 
    FindClusters(resolution = 0.5)


Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  TRBC2, TRAC, IL32, IL7R, LTB, CD7, CD247, ISG20, TRBC1, TCF7 
	   GZMM, CD27, MALAT1, CCR7, SYNE2, LEF1, NOSIP, GZMA, CTSW, CST7 
	   CCL5, NKG7, MAL, SAMD3, TRAT1, RGCC, CD8B, PRF1, CD8A, KLRD1 
Negative:  FCN1, CST3, MNDA, LYZ, S100A9, LST1, FGL2, CSTA, AC020656.1, SERPINA1 
	   CTSS, CFD, AIF1, VCAN, S100A8, TYMP, CYBB, FTL, NCF2, CD68 
	   MS4A6A, PSAP, CD14, GRN, SPI1, TYROBP, IGSF6, MPEG1, FTH1, S100A12 
PC_ 2 
Positive:  NKG7, CST7, GZMA, KLRD1, PRF1, CCL5, S100A4, CTSW, ANXA1, GZMM 
	   FGFBP2, PFN1, GNLY, GZMH, CCL4, SRGN, EFHD2, IL32, CD247, HOPX 
	   TMSB4X, IFITM2, ITGB2, GZMB, MATK, SPON2, SAMD3, KLRG1, KLRF1, FCGR3A 
Negative:  CD79A, MS4A1, IGHM, BANK1, CD79B, IGHD, RALGPS2, LINC00926, TCL1A, IGKC 
	   VPREB3, TNFRSF13C, SPIB, BCL11A, HLA-DRA, FAM129C, HLA-DQA2, HLA-DQA1, FCER2, LINC02397 
	   AFF3, HLA-DQB1, FCRL1, CD74, HVCN1, SWAP70, CD22, MEF2C,

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 59011
Number of edges: 1810948

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9295
Number of communities: 19
Elapsed time: 31 seconds


In [12]:
flex_fully <- NormalizeData(flex_fully) %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>% 
    RunUMAP(dims = 1:20) %>% 
    FindNeighbors(dims = 1:20) %>% 
    FindClusters(resolution = 0.5)


Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  LTB, TRBC1, KLRK1, CCR7, PIM2, PYHIN1, CD27, GRAP2, LEF1, PRF1 
	   IL2RB, KLRG1, CD28, GZMA, CST7, CTSW, TRAT1, MAL, NKG7, CCL5 
	   HIST1H1D, CD8A, KLRD1, NELL2, MATK, TGFBR3, TBX21, MYBL1, HIST1H1C, OBSCN 
Negative:  NCF2, CD68, SERPINA1, CLEC7A, MNDA, MEFV, LILRB2, CST3, CSF3R, EMILIN2 
	   IFI30, TNFAIP2, TYMP, LRRC25, SPI1, HCK, LRP1, TGFBI, KLF4, PLXNB2 
	   VCAN, KCTD12, CD14, TIMP2, S100A9, ZNF385A, FPR1, FGL2, LYZ, HK3 
PC_ 2 
Positive:  PRF1, NKG7, CST7, GZMA, ANXA1, SRGN, SYNE1, CCL5, IL2RB, KLRD1 
	   CX3CR1, CTSW, ADGRG1, KLRG1, FGFBP2, FCRL6, TBX21, PYHIN1, MATK, S1PR5 
	   TGFBR3, GNLY, KLRK1, DOK2, ID2, GZMH, MYBL1, TNFRSF1A, CCL4, C1orf21 
Negative:  CD79A, MS4A1, BANK1, NIBAN3, CD22, PAX5, FCRL1, IGHM, TNFRSF13C, FCRL2 
	   BCL11A, CD79B, IGHD, RALGPS2, SPIB, RUBCNL, POU2AF1, WDFY4, BLK, FCRLA 
	   BLNK, CD24, SWAP70, FCER2, AFF3, IGKC, CD19, TC

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 39465
Number of edges: 1353190

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9366
Number of communities: 22
Elapsed time: 15 seconds


# Label Transfer

In [14]:
ref <- LoadH5Seurat(file = '/home/jupyter/pbmc_multimodal.h5seurat')

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding variable feature information for ADT

Adding miscellaneous information for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for w

In [ ]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = v3_fully,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
)
v3_fully <- TransferData(anchorset = anchors, reference = ref, query = v3_fully,
                refdata = list(
                  celltype.l1 = "celltype.l1",
                  celltype.l2 = "celltype.l2",
                  celltype.l3 = "celltype.l3",
                  predicted_ADT = 'ADT'))

In [ ]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = flex_fully,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
)
flex_fully <- TransferData(anchorset = anchors, reference = ref, query = flex_fully,
                refdata = list(
                  celltype.l1 = "celltype.l1",
                  celltype.l2 = "celltype.l2",
                  celltype.l3 = "celltype.l3",
                  predicted_ADT = 'ADT'))

In [ ]:
v3_fully$sample   <- substr(start = 1, stop = 7,v3_fully$orig.ident)
flex_fully$sample <- substr(start = 1, stop = 7,flex_fully$orig.ident)

v3_fully$sample   <- gsub('PB00607', 'Donor 1', v3_fully$sample)
v3_fully$sample   <- gsub('PB00609', 'Donor 2', v3_fully$sample)
v3_fully$sample   <- gsub('PB00610', 'Donor 3', v3_fully$sample)

flex_fully$sample <- gsub('PB00607', 'Donor 1', flex_fully$sample)
flex_fully$sample <- gsub('PB00609', 'Donor 2', flex_fully$sample)
flex_fully$sample <- gsub('PB00610', 'Donor 3', flex_fully$sample)


In [ ]:
v3_fully$sample_tech <- '3Prime'
flex_fully$sample_tech <- 'Flex'


In [ ]:
celltype_prop <- function(so,lvl,metadata){
    metadata_df <- so@meta.data

    if (lvl == 1){counts <- dplyr::count(metadata_df, predicted.celltype.l1, eval(parse(text=metadata)))}
    if (lvl == 2){counts <- dplyr::count(metadata_df, predicted.celltype.l2, eval(parse(text=metadata)))}
    names(counts)[2] <- metadata
    
    sum_counts <- metadata_df %>% dplyr::count(eval(parse(text=metadata)))
    names(sum_counts)[1] <- metadata

    joined_counts <- left_join(counts, sum_counts, by = metadata)

    joined_counts$perc <- joined_counts$n.x/joined_counts$n.y
    return(joined_counts)
    }

In [ ]:
ct_v3.no.other   <- v3_fully   %>% subset(!(subset = predicted.celltype.l1 == 'other')) %>% celltype_prop(1,'sample')
head(ct_v3.no.other)

ct_flex.no.other <- flex_fully %>% subset(!(subset = predicted.celltype.l1 == 'other')) %>% celltype_prop(1,'sample')
head(ct_flex.no.other)

In [ ]:
joined_counts.no.other <- left_join(ct_v3.no.other,ct_flex.no.other,by = c('predicted.celltype.l1', 'sample'))


In [ ]:
joined_counts.no.other$predicted.celltype.l1 <- factor(
    joined_counts.no.other$predicted.celltype.l1, 
    levels = c("CD4 T", "CD8 T", "other T", "B", "NK", "Mono", "DC")
)

In [ ]:
options(repr.plot.width=10,repr.plot.height=8) 


p <- ggplot(joined_counts.no.other, aes(x = perc.x, y = perc.y, color = predicted.celltype.l1, shape = sample)) +
     geom_abline(slope = 1, linetype = 'dashed', size = 0.5) +
     geom_smooth(inherit.aes = FALSE, aes(x = perc.x, y = perc.y), method='lm', se = FALSE, color = "black") + #, linewidth = 0.5) +
     geom_point(size = 3) + 
     # scale_color_manual(values = DiscretePalette_scCustomize(num_colors = 8,palette = "varibow")) +
     scale_x_log10(labels = scales::percent, limits = c(0.01,0.85)) +
     scale_y_log10(labels = scales::percent, limits = c(0.01,0.85)) + 
     # stat_cor(inherit.aes = FALSE, aes(x = flex.perc, y = v3.perc), size = 5, 
     #          r.accuracy = 0.001, p.accuracy = 0.001, show.legend = FALSE) + 
     xlab("PBMC Population Frequency") +
     ylab("CWB Population Frequency") +
     theme_bw() +    
     theme(
           axis.line = element_line(color='black'),
           panel.grid.major = element_blank(),
           panel.grid.minor = element_blank(),
           axis.text.x=element_text(colour="black"),
           axis.text.y=element_text(colour="black"),
           text = element_text(family="sans", size = 12),
           plot.margin = margin(0.2,0.2,0.2,0.2, "cm"),
          )

p

In [ ]:
v3_fully   %>% saveRDS('/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/B077_3prime/Fig2BC_v3_fully.rds')
flex_fully %>% saveRDS('/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/exp340/Fig2BC_flex_fully.rds')

In [ ]:
sessionInfo()